# Chapter 3: Calculating the similarity between documents

Continue here - I need to make a decision about whether this is in the same chapter with tf-idf or not. Probably as otherwise the end of that chapter will involve manually calculating the tf-idf for every word, as well as being a bit boring and anti-climatic.

In [1]:
# Need to find a good step-by-step tutorial of constructing the tf-idf matrix by hand
# Then I need to find another step-by-step of the cosine similarity
# I should work through both of those completely before I do the chapter!! :D
# Also, I can do the graph demonstrating the document-term similarity with the tf-idf!

In [1]:
documents = (
    "The sky is blue",
    "The sun is bright",
    "The sun in the sky is bright",
    "We can see the shining sun, the bright sun"
)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

vectorizer.fit_transform(documents)
print vectorizer.vocabulary

None


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print tfidf_matrix.shape

(4, 11)


In [13]:
import operator
from pprint import pprint

sorted(tfidf_vectorizer.vocabulary_.items(), key=operator.itemgetter(1))

[(u'blue', 0),
 (u'bright', 1),
 (u'can', 2),
 (u'in', 3),
 (u'is', 4),
 (u'see', 5),
 (u'shining', 6),
 (u'sky', 7),
 (u'sun', 8),
 (u'the', 9),
 (u'we', 10)]

In [16]:
print tfidf_matrix.todense()

[[ 0.65919112  0.          0.          0.          0.42075315  0.          0.
   0.51971385  0.          0.34399327  0.        ]
 [ 0.          0.52210862  0.          0.          0.52210862  0.          0.
   0.          0.52210862  0.42685801  0.        ]
 [ 0.          0.3218464   0.          0.50423458  0.3218464   0.          0.
   0.39754433  0.3218464   0.52626104  0.        ]
 [ 0.          0.23910199  0.37459947  0.          0.          0.37459947
   0.37459947  0.          0.47820398  0.39096309  0.37459947]]


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)

array([[ 1.        ,  0.36651513,  0.52305744,  0.13448867]])

In [47]:
dm = tfidf_matrix.todense()
dm

matrix([[ 0.65919112,  0.        ,  0.        ,  0.        ,  0.42075315,
          0.        ,  0.        ,  0.51971385,  0.        ,  0.34399327,
          0.        ],
        [ 0.        ,  0.52210862,  0.        ,  0.        ,  0.52210862,
          0.        ,  0.        ,  0.        ,  0.52210862,  0.42685801,
          0.        ],
        [ 0.        ,  0.3218464 ,  0.        ,  0.50423458,  0.3218464 ,
          0.        ,  0.        ,  0.39754433,  0.3218464 ,  0.52626104,
          0.        ],
        [ 0.        ,  0.23910199,  0.37459947,  0.        ,  0.        ,
          0.37459947,  0.37459947,  0.        ,  0.47820398,  0.39096309,
          0.37459947]])

Ex: Find the similarity between documents 1 and 2.  
d1 = (5, 0, 3, 0, 2, 0, 0, 2, 0, 0)  
d2 = (3, 0, 2, 0, 1, 1, 0, 1, 0, 1)  
d1 ⋅ d2 = 5*3+0*0+3*2+0*0+2*1+0*1+0*1+2*1+0*0+0*1 = 25  
||d1|| = (5*5+0*0+3*3+0*0+2*2+0*0+0*0+2*2+0*0+0*0)0.5 = (42) 0.5 = 6.481  
||d2|| = (3*3+0*0+2*2+0*0+1*1+1*1+0*0+1*1+0*0+1*1)0.5 = (17) 0.5 = 4.12  
cos(d1, d2 ) = 0.94

In [49]:
vec_dot_prod = dm.item(0, 0) * dm.item(2, 0) + dm.item(0, 1) * dm.item(2, 1) +\
               dm.item(0, 2) * dm.item(2, 2) + dm.item(0, 3) * dm.item(2, 3) +\
               dm.item(0, 4) * dm.item(2, 4) + dm.item(0, 5) * dm.item(2, 5) +\
               dm.item(0, 6) * dm.item(2, 6) + dm.item(0, 7) * dm.item(2, 7) +\
               dm.item(0, 8) * dm.item(2, 8) + dm.item(0, 9) * dm.item(2, 9) +\
               dm.item(0, 10) * dm.item(2, 10)

In [50]:
vec_dot_prod

0.5230574383703659

In [51]:
v1_sq = (dm.item(0, 0) * dm.item(0, 0) + dm.item(0, 1) * dm.item(0, 1) +\
         dm.item(0, 2) * dm.item(0, 2) + dm.item(0, 3) * dm.item(0, 3) +\
         dm.item(0, 4) * dm.item(0, 4) + dm.item(0, 5) * dm.item(0, 5) +\
         dm.item(0, 6) * dm.item(0, 6) + dm.item(0, 7) * dm.item(0, 7) +\
         dm.item(0, 8) * dm.item(0, 8) + dm.item(0, 9) * dm.item(0, 9) +\
         dm.item(0, 10) * dm.item(0, 10)) * 0.5

In [52]:
v2_sq = (dm.item(2, 0) * dm.item(2, 0) + dm.item(2, 1) * dm.item(2, 1) +\
         dm.item(2, 2) * dm.item(2, 2) + dm.item(2, 3) * dm.item(2, 3) +\
         dm.item(2, 4) * dm.item(2, 4) + dm.item(2, 5) * dm.item(2, 5) +\
         dm.item(2, 6) * dm.item(2, 6) + dm.item(2, 7) * dm.item(2, 7) +\
         dm.item(2, 8) * dm.item(2, 8) + dm.item(2, 9) * dm.item(2, 9) +\
         dm.item(2, 10) * dm.item(2, 10)) * 0.5

In [53]:
v1_sq

0.4999999999999999

In [54]:
v2_sq

0.49999999999999983

In [55]:
vec_dot_prod / v1_sq * v2_sq

0.5230574383703658

## Define the corpus

Let's put together an example set, but this would also include all of the fairytales as well.

In [131]:
documents = [
    'The princess was clever',
    'The prince was handsome',
    'The prince loved the clever princess',
    '"Prince, handsome prince", said the princess',
]

## Create the index vocabulary

Index vocabulary is simply a numbered list of 'features' (i.e., terms) in our corpus. It includes (unless we explicitly define it) every word in the corpus, hence another reason to get rid of stop words early. These will also be the columns on the term-document matrix. 

In [70]:
index_vocabulary = {
    1: 'clever',
    2: 'handsome',
    3: 'loved',
    4: 'prince',
    5: 'princess',
    6: 'said'
}

In [72]:
index_vocabulary

{1: 'clever', 2: 'handsome', 3: 'loved', 4: 'prince', 5: 'princess', 6: 'said'}

## Turning documents into vectors

To turn a document into a vector, we simply need to count the number of time each term in the index vocabulary occurs in that document. Note that I need to strip out the punctuation and convert the words to lowercase. For the first document (sentence) in the corpus, we get:

In [146]:
import string

def count(document_list, term):
    doc = document_list.lower().translate(None, string.punctuation).split()
    match = [x for x in doc if x == term] 
    return len(match)

In [149]:
documents[0]

'The princess was clever'

In [147]:
m1 = [count(documents[0], x) for x in index_vocabulary.values()]
m1

[1, 0, 0, 0, 1, 0]

We can also represent this a bit more explicitly with a labelled dataframe.

In [163]:
from pandas import DataFrame
df = DataFrame(m1).T
df.columns = index_vocabulary.values()
df

,clever,handsome,loved,prince,princess,said
0,1,0,0,0,1,0


In [180]:
import numpy as np

np.array(tuple([count(d, x) for x in index_vocabulary.values()] for d in documents))

array([[1, 0, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [0, 1, 0, 2, 1, 1]])

In [182]:
DataFrame(np.array(tuple([count(d, x) for x in index_vocabulary.values()] for d in documents)),
          columns = index_vocabulary.values())

,clever,handsome,loved,prince,princess,said
0,1,0,0,0,1,0
1,0,1,0,1,0,0
2,1,0,1,1,1,0
3,0,1,0,2,1,1


In [63]:
from sklearn.feature_extraction.text import CountVectorizer
import operator
from pprint import pprint
vectorizer = CountVectorizer(stop_words = 'english')

vectorizer.fit_transform(documents)
sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))

[(u'clever', 0),
 (u'handsome', 1),
 (u'loved', 2),
 (u'prince', 3),
 (u'princess', 4),
 (u'said', 5)]

In [68]:
smatrix = vectorizer.transform(documents)
print smatrix

  (0, 0)	1
  (0, 4)	1
  (1, 1)	1
  (1, 3)	1
  (2, 0)	1
  (2, 2)	1
  (2, 3)	1
  (2, 4)	1
  (3, 1)	1
  (3, 3)	2
  (3, 4)	1
  (3, 5)	1


In [181]:
smatrix.todense()

matrix([[1, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0, 0],
        [1, 0, 1, 1, 1, 0],
        [0, 1, 0, 2, 1, 1]])

In [185]:
DataFrame(smatrix.todense(), columns = index_vocabulary.values())

,clever,handsome,loved,prince,princess,said
0,1,0,0,0,1,0
1,0,1,0,1,0,0
2,1,0,1,1,1,0
3,0,1,0,2,1,1
